# NLP Toxcicty Machine Learning Model

## Imports

In [1]:
import pandas as pd
import string

from nltk.corpus import stopwords, wordnet
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multioutput import MultiOutputClassifier

import re

## Data

In [4]:
df_train= pd.read_csv('data/train.csv', index_col = 'id')

In [5]:
df_train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   comment_text   159571 non-null  object
 1   toxic          159571 non-null  int64 
 2   severe_toxic   159571 non-null  int64 
 3   obscene        159571 non-null  int64 
 4   threat         159571 non-null  int64 
 5   insult         159571 non-null  int64 
 6   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 9.7+ MB


In [7]:
df_test = pd.read_csv('data/test_labels.csv')

In [8]:
df_test.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [9]:
# Function for improving parts of speech information

### get_wordnet_pos was taken from Lecture 51-nlp_modeling.ipynb 
### link to the lecture: https://github.com/dvdhartsman/NTL-DS-080723/blob/main/4phase/51-nlp_modeling.ipynb


def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [10]:
# Function for handling the transformation of data

### preprocess taken from nlp-sentiment-analysis
### link to the project: https://github.com/dvdhartsman/NLP-Sentiment-Analysis/blob/main/Text_Classification_Final_Notebook.ipynb

def preprocess(comment):
    """
    This is a function that is intended to handle all of the tokenization, lemmatization, and other
    preprocessing for our tweet data. It will make use of objects from other libraries, and will return
    a complete list of tokens that are ready to be vectorized into numerical data.
    """
    
    # Create a list of stopwords to be removed from our tokenized word list
    stops = stopwords.words("english")
    # Add punctuation to the list of stopwords
    stops += string.punctuation
    # Providing a regex pattern for the tokenizer to handle
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    # Instantiating a tokenizer
    tokenizer = RegexpTokenizer(pattern)
    # Creating a list of raw tokens
    raw_tokens = tokenizer.tokenize(comment)
    # Using a comprehension to lower case every token
    lower_tokens = [i.lower() for i in raw_tokens]
    # Remove the stopwords from the list of tokens
    stopped_words = [i for i in lower_tokens if i not in stops]
    
    # Adding parts of speech to prepare for Lemmatization
    
    # This is the initial method to get parts of speech
    stopped_words = pos_tag(stopped_words)
    
    # Get_wordnet_pos() is the function to modify the pos definitions/assignments, creates tuples of (<word>, <pos>)
    stopped_words = [(word[0], get_wordnet_pos(word[1])) for word in stopped_words]
    
    lemmatizer = WordNetLemmatizer() 
    
    # This corrects the parts of speech and maximizes the usefulness of the lemmatization!!!!!
    document = [lemmatizer.lemmatize(word[0], word[1]) for word in stopped_words]
    
    # Re-join the list of cleaned tokens
    cleaned_doc = " ".join(document)
    return cleaned_doc

In [11]:
X = df_train.comment_text
y = df_train[['toxic', 'severe_toxic', 'obscene', 'threat','insult','identity_hate']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify=['toxic', 'severe_toxic','identity_hate'])
y_toxic_train = df_train.toxic
y_severe_toxic_train = df_train.severe_toxic
y_obscene_train = df_train.obscene
y_threat_train = df_train.threat
y_insult_train = df_train['identity_hate']

In [12]:
preprocess(X.iloc[1])

"d'aww match background colour i'm seemingly stuck thanks talk january utc"

In [13]:
X_clean = X.apply(preprocess)

KeyboardInterrupt: 

In [ ]:
X_clean.to_csv()

In [86]:
count_vec = CountVectorizer(ngram_range=(1, 2), max_features=10000)
tf_vec = TfidfVectorizer(ngram_range=(1, 2), max_features=10000)

In [87]:
count_vec.fit(X_clean)
tf_vec.fit(X_clean)

TfidfVectorizer(max_features=10000, ngram_range=(1, 2))

In [88]:
X_count = count_vec.transform(X_clean)
X_tfidf = tf_vec.transform(X_clean)

In [89]:
X_count

<159571x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 4444977 stored elements in Compressed Sparse Row format>

In [90]:
X_tfidf

<159571x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 4444977 stored elements in Compressed Sparse Row format>

In [91]:
X_train_cv, X_test_cv, y_train, y_test = train_test_split(X_count,y, random_state = 42)

In [92]:
X_train_tf, X_test_tf, y_train, y_test = train_test_split(X_tfidf,y, random_state = 42)

In [93]:
y_train.value_counts()

toxic  severe_toxic  obscene  threat  insult  identity_hate
0      0             0        0       0       0                107512
1      0             0        0       0       0                  4309
                     1        0       1       0                  2833
                                      0       0                  1309
                     0        0       1       0                   893
       1             1        0       1       0                   731
       0             1        0       1       1                   470
0      0             1        0       0       0                   237
                     0        0       1       0                   213
1      1             1        0       1       1                   193
0      0             1        0       1       0                   141
1      1             1        0       0       0                   124
       0             0        0       1       1                   106
                     1        

In [94]:
#X_train.apply(preprocess)

"""id
0000997932d777bf    explanation edits make username hardcore metal...
000103f0d9cfb60f    d'aww match background colour i'm seemingly st...
000113f07ec002fd    hey man i'm really try edit war guy constantly...
0001b41b1c6bb37e    can't make real suggestion improvement wonder ...
0001d958c54c6e35                 sir hero chance remember page that's
                                          ...                        
ffe987279560d7ff    second time ask view completely contradict cov...
ffea4adeee384e90                 ashamed horrible thing put talk page
ffee36eab5c267c9    spitzer umm there actual article prostitution ...
fff125370e4aaaf3    look like actually put speedy first version de...
fff46fc426af1f9a    really think understand come idea bad right aw...
Name: comment_text, Length: 159571, dtype: object"""

"id\n0000997932d777bf    explanation edits make username hardcore metal...\n000103f0d9cfb60f    d'aww match background colour i'm seemingly st...\n000113f07ec002fd    hey man i'm really try edit war guy constantly...\n0001b41b1c6bb37e    can't make real suggestion improvement wonder ...\n0001d958c54c6e35                 sir hero chance remember page that's\n                                          ...                        \nffe987279560d7ff    second time ask view completely contradict cov...\nffea4adeee384e90                 ashamed horrible thing put talk page\nffee36eab5c267c9    spitzer umm there actual article prostitution ...\nfff125370e4aaaf3    look like actually put speedy first version de...\nfff46fc426af1f9a    really think understand come idea bad right aw...\nName: comment_text, Length: 159571, dtype: object"

# Modeling

In [95]:
class MultiOutput():
    # a class to more easily test, interpret and store different classifiers for a multioutput dataset
    model_list = []
    model_df = pd.DataFrame(columns = ['Classifier', 'train_accuracy','train_prec','train_recall','train_f1',
                                      'test_accuracy','test_prec','test_recall','test_f1'])
    
    def __init__(self, name, clf, X_train, X_test, y_train, y_test):
        self.name = name
        self.clf = classifier
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        # Measuring model
        self.train_results = cross_validate(self.model, self.X_train, self.y_train, scoring=[
            'precision', 'accuracy', 'recall', 'f1', 'neg_log_loss'], n_jobs=4, verbose=1)
        

## Baseline Dummy 

In [96]:
dummy = DummyClassifier(strategy='most_frequent')

In [97]:
dummy_clf = MultiOutputClassifier(dummy).fit(X_train_cv,y_train)

In [98]:
accuracy_score(y_train, dummy_clf.predict(X_train_cv))

0.898343889436655

In [99]:
accuracy_score(y_test, dummy_clf.predict(X_test_cv))

0.8982528263103803

In [100]:
y_train

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
c912439967ba8a35,0,0,0,0,0,0
b312f612d3394d5b,0,0,0,0,0,0
813c210bf7f27377,0,0,0,0,0,0
c231999bc75dcd9e,0,0,0,0,0,0
d60a1de68cf593c1,0,0,0,0,0,0
...,...,...,...,...,...,...
811ed72c51830f42,0,0,0,0,0,0
2acc7c7d0386401f,0,0,0,0,0,0
c1f95b89050a9ee4,1,0,0,0,0,0


## Logistic Regression

In [101]:
logreg_clf = MultiOutputClassifier(LogisticRegression()).fit(X_train_cv, y_train)

C:\Users\rchag\anaconda3\envs\learn-env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\rchag\anaconda3\envs\learn-env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [102]:
accuracy_score(y_train, logreg_clf.predict(X_train_cv))

0.9318755326793563

In [103]:
accuracy_score(y_test, logreg_clf.predict(X_test_cv))

0.9146466798686487

In [104]:
logreg_clf.predict(X_test_cv)

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [105]:
logreg_clf_tf = MultiOutputClassifier(LogisticRegression()).fit(X_train_tf, y_train)

In [106]:
accuracy_score(y_train, logreg_clf_tf.predict(X_train_tf))

0.9246227376794398

In [107]:
accuracy_score(y_test, logreg_clf_tf.predict(X_test_tf))

0.9191336826009575

## Random Forest

In [109]:
from sklearn.ensemble import RandomForestClassifier

rfclf = MultiOutputClassifier(RandomForestClassifier(n_jobs = 1)).fit(X_train_cv, y_train)

In [110]:
accuracy_score(y_test, rfclf.predict(X_test_cv))

0.9125410473015316

In [111]:
rfclf_tf = MultiOutputClassifier(RandomForestClassifier(n_jobs = 1)).fit(X_train_tf, y_train)

In [112]:
accuracy_score(y_test, rfclf_tf.predict(X_test_tf))

0.9163512395658386

## Multinomial Naive Bayes

In [113]:
from sklearn.naive_bayes import MultinomialNB

mnb_cv = MultiOutputClassifier(MultinomialNB()).fit(X_train_cv, y_train)

In [114]:
accuracy_score(y_test, mnb_cv.predict(X_test_cv))

0.9033915724563206

In [115]:
mnb_tf = MultiOutputClassifier(MultinomialNB()).fit(X_train_tf, y_train)

In [116]:
accuracy_score(y_test, mnb_tf.predict(X_test_cv))

0.8979018875491941

In [157]:
mnb_tf.predict(X_test_cv)

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

## Sequential

In [14]:
from keras.models import Sequential
from keras import layers

ModuleNotFoundError: No module named 'keras'

In [5]:
import tensorflow as tf



C:\Users\rchag\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:585: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [118]:
def get_model(n_inputs, n_outputs):    
    model = Sequential()
    model.add(layers.Dense(128, input_dim = n_inputs, activation = 'relu'))
    model.add(layers.Dense(128, activation = 'relu'))
    model.add(layers.Dense(n_outputs, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam')
    return model

In [119]:
nn1 = get_model(10000, y_train.shape[1])

In [129]:
X_train_cv_df = pd.DataFrame(X_train_cv.toarray())

In [130]:
X_train_cv_df.head()


,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
nn1.fit(X_train_cv_df, y_train, verbose = 1, epochs = 10)

Epoch 1/10
3740/3740 [==============================] - 16s 4ms/step - loss: 0.0239
Epoch 2/10
3740/3740 [==============================] - 16s 4ms/step - loss: 0.0174
Epoch 3/10
3740/3740 [==============================] - 16s 4ms/step - loss: 0.0136
Epoch 4/10
3740/3740 [==============================] - 16s 4ms/step - loss: 0.0110
Epoch 5/10
3740/3740 [==============================] - 16s 4ms/step - loss: 0.0087
Epoch 6/10
3740/3740 [==============================] - 16s 4ms/step - loss: 0.0081
Epoch 7/10
3740/3740 [==============================] - 16s 4ms/step - loss: 0.0065
Epoch 8/10
3740/3740 [==============================] - 16s 4ms/step - loss: 0.0060
Epoch 9/10
3740/3740 [==============================] - 16s 4ms/step - loss: 0.0056
Epoch 10/10
3740/3740 [==============================] - 16s 4ms/step - loss: 0.0057


In [163]:
nn1.predict(X_test_cv)

array([[1.1207763e-07, 1.2251892e-21, 9.7162364e-18, 1.1926257e-18,
        2.3212488e-16, 4.7794572e-08],
       [6.2049849e-20, 4.8000025e-27, 1.8291317e-27, 1.3207786e-22,
        1.9511895e-20, 1.6348332e-19],
       [9.0539538e-07, 3.4379249e-13, 9.7092789e-07, 6.2708168e-12,
        2.9626599e-07, 7.9645686e-11],
       ...,
       [1.1720364e-34, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.1772889e-16, 6.2701701e-31, 1.5096666e-21, 1.3533453e-27,
        1.5048285e-17, 1.5146988e-11],
       [4.5478344e-04, 9.2410046e-10, 3.8160433e-06, 4.9743015e-10,
        8.4841361e-09, 5.4978336e-06]], dtype=float32)

In [164]:
print(nn1.predict(X_test_cv[1]))

[[6.2050088e-20 4.8000025e-27 1.8291177e-27 1.3207786e-22 1.9511895e-20
  1.6348457e-19]]


In [168]:
yhat_cv = nn1.predict(X_test_cv)

In [169]:
yhat_cv

array([[1.1207763e-07, 1.2251892e-21, 9.7162364e-18, 1.1926257e-18,
        2.3212488e-16, 4.7794572e-08],
       [6.2049849e-20, 4.8000025e-27, 1.8291317e-27, 1.3207786e-22,
        1.9511895e-20, 1.6348332e-19],
       [9.0539538e-07, 3.4379249e-13, 9.7092789e-07, 6.2708168e-12,
        2.9626599e-07, 7.9645686e-11],
       ...,
       [1.1720364e-34, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.1772889e-16, 6.2701701e-31, 1.5096666e-21, 1.3533453e-27,
        1.5048285e-17, 1.5146988e-11],
       [4.5478344e-04, 9.2410046e-10, 3.8160433e-06, 4.9743015e-10,
        8.4841361e-09, 5.4978336e-06]], dtype=float32)

In [171]:
yhat_cv = yhat_cv.round()

In [172]:
yhat_cv[1]

array([0., 0., 0., 0., 0., 0.], dtype=float32)

In [173]:
accuracy_score(y_test, yhat_cv)

0.8992805755395683

In [174]:
yhat_train_cv = nn1.predict(X_train_cv)

In [175]:
yhat_train_cv = yhat_train_cv.round()

In [176]:
accuracy_score(y_train, yhat_train_cv)

0.9938835876267986

In [187]:
nn2 = get_model(300, y_train.shape[1])

In [188]:
X_train_cv_300 = X_train_cv_df.iloc[:,0:300]

In [189]:
X_train_cv_300.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [190]:
nn2.fit(X_train_cv_300, y_train, verbose = 1, epochs = 10)

Epoch 1/10
3740/3740 [==============================] - 2s 519us/step - loss: 0.1438
Epoch 2/10
3740/3740 [==============================] - 2s 514us/step - loss: 0.1360
Epoch 3/10
3740/3740 [==============================] - 2s 511us/step - loss: 0.1348
Epoch 4/10
3740/3740 [==============================] - 2s 516us/step - loss: 0.1342
Epoch 5/10
3740/3740 [==============================] - 2s 510us/step - loss: 0.1339
Epoch 6/10
3740/3740 [==============================] - 2s 509us/step - loss: 0.1334
Epoch 7/10
3740/3740 [==============================] - 2s 512us/step - loss: 0.1331
Epoch 8/10
3740/3740 [==============================] - 2s 509us/step - loss: 0.1327
Epoch 9/10
3740/3740 [==============================] - 2s 520us/step - loss: 0.1328
Epoch 10/10
3740/3740 [==============================] - 2s 511us/step - loss: 0.1320


In [192]:
yhat_train_cv_2 = nn2.predict(X_train_cv_300)

In [193]:
yhat_train_cv_2 = yhat_train_cv_2.round()

In [194]:
accuracy_score(y_train, yhat_train_cv_2)

0.8989120807500125